# <center>Aprendizaje Automático Avanzado: Ejercicio final</center>
### <center>Autor: Julen Ercibengoa</center>

## 1. Contexto

Actualmente estoy trabajando en la empresa **Tekniker** la cual está involucrada en el proyecto GAIA. Este es un proyecto que pretende dar apoyo a los servicios de extinción de incendios forestales mediante herramientas de prevención y detección temprana utilizando la Inteligencia Artificial.

Mi trabajo consiste en crear un modelo de Aprendizaje Automático que prediga el riesgo de incendio forestal a nivel nacional diariamente. Es decir, el objetivo final del trabajo (el cual es mi Trabajo de Fin de Máster), es generar un modelo que con los datos disponibles hasta hoy, genere un mapa de mañana de riesgo de incendio forestal en toda España. 

Para conseguir ese objetivo hacen falta varios pasos, y este trabajo será uno de ellos. En este trabajo haremos una primera prueba y generaremos un predictor de riesgo de incendio forestal que prediga el riesgo de incendio forestal mensual. Además, haremos varios análisis para decidir qué variable son relevantes para el modelo.

## 2. Bases de Datos

Para predecir el riesgo de incendio forestal hay que generar una base de datos con la cual entrenar un modelo de Inteligencia Artificial. En el estado del arte [(Ver el análisis del estado del arte)](SOTA.pdf) se sigue una metodología común que vamos a utilizar: se divide la región a analizar en celdas espacio-temporales y cada celda tiene asignadas variables tabulares. Para generar dicha base de datos, vamos a utilizar las siguientes fuentes de datos:
- <u>Datos históricos de fuegos</u>: base de datos de [EFFIS](https://forest-fire.emergency.copernicus.eu/)
- <u>Datos del tipo de terreno</u>: base de datos [Corine Land Cover](https://land.copernicus.eu/en/products/corine-land-cover)
- <u>Datos DEM (Digital Elevation Model)</u>: base de datos de [EU DEM](https://portal.opentopography.org/raster?opentopoID=OTSDEM.032021.4326.3&minX=-17.18261718749999&minY=31.097629956393973&maxX=15.952148437499998&maxY=45.43508099838451)
- <u>Distancia a carreteras</u>, <u>distancia a rios</u> y <u>densidad poblacional</u>: [WorldPop](https://www.worldpop.org/)
- <u>Distancia a vías de tren</u>: [OpenStreetMap](https://www.openstreetmap.org/#map=6/40.01/-2.49)
- <u>Proporción de natura2000</u>: [Gobierno de España](https://www.miteco.gob.es/es/biodiversidad/servicios/banco-datos-naturaleza/informacion-disponible/rednatura_2000_desc.html)

Aunque en las visualizaciones de los datos que vamos a hacer a continuación no aparezca, el área que vamos a analizar sera toda España, incluyendo las Islas Canarias.

### 2.1 Datos históricos de fuegos: EFFIS

El primer conjunto de datos que vamos a utilizar serán los datos históricos de EFFIS (European Forest Fire Information System). Este conjunto de datos guarda la información de cuándo inició el fuego, cuándo finalizó, el área que quemó y la forma que tiene junto a dónde pasó. Los datos que tienen recogidos van desde 2008 hasta la actualidad y usaremos solo los incendios de más de 5 hectáreas, con un total de 5862 fuegos.

Para visualizar el conjunto de datos utilizaremos el paquete **geopandas**, que permite visualizar los datos con información geográfica.

In [28]:
import geopandas as gpd
gdf_fires = gpd.read_file("../data/Variables/EFFIS_historical_postprocessed.json")
gdf_fires.head()

,initialdate,finaldate,area_ha,geometry
0,2008-04-27,2008-04-27,419,"MULTIPOLYGON (((1652130.767 1048788.051, 16513..."
1,2008-06-17,2008-06-17,104,"MULTIPOLYGON (((3252074.625 1640344.173, 32521..."
2,2008-06-19,2008-06-19,523,"MULTIPOLYGON (((2906844.761 1591397.039, 29068..."
3,2008-07-01,2008-07-01,253,"MULTIPOLYGON (((3016594.759 1855823.362, 30161..."
4,2008-07-07,2008-07-07,58,"MULTIPOLYGON (((3115823.664 1790836.728, 31160..."


In [29]:
import contextily as ctx
import matplotlib.pyplot as plt
gdf_fires = gdf_fires.to_crs(epsg=3857)
minx, miny, maxx, maxy = gdf_fires.total_bounds
ax = gdf_fires.plot(column="area_ha", legend=True, figsize=(12,12), cmap="plasma", legend_kwds={'label': "Área quemada (ha)", 'orientation': "horizontal", 'shrink': 0.5, 'aspect': 50, 'pad': 0.01})
ax.set_xlim([-1.25*10**6, maxx])
ax.set_ylim([4.25*10**6, maxy])
ctx.add_basemap(ax, crs=gdf_fires.crs.to_string())
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Área quemada en España (2008-2024)", fontsize=20, pad=20)
plt.savefig("../data/figures/fires_plot.png", bbox_inches="tight", dpi=300)
plt.clf()


<Figure size 1200x1200 with 0 Axes>

<div style="text-align: center;">
    <img src="../data/figures/fires_plot.png" alt="Área quemada en España (2008-2024)" width="800">
</div>

### 2.2 Datos del tipo de terreno: Corine Land Cover

El conjunto de datos Corine Land Cover divide a toda Europa en celdas de 100m $\times$ 100m y asigna a cada celda una clase de las 44 clases que tienen definidas. Las clases son las siguientes: 


\begin{array}{|c|c|c|c|}
\hline
\textbf{Class} & \textbf{LABEL1} & \textbf{LABEL2} & \textbf{LABEL3} \\
\hline
1 & \text{Artificial surfaces} & \text{Urban fabric} & \text{Continuous urban fabric} \\
\hline
2 & \text{Artificial surfaces} & \text{Urban fabric} & \text{Discontinuous urban fabric} \\
\hline
3 & \text{Artificial surfaces} & \text{Industrial, commercial and transport units} & \text{Industrial or commercial units} \\
\hline
4 & \text{Artificial surfaces} & \text{Industrial, commercial and transport units} & \text{Road and rail networks and associated land} \\
\hline
5 & \text{Artificial surfaces} & \text{Industrial, commercial and transport units} & \text{Port areas} \\
\hline
6 & \text{Artificial surfaces} & \text{Industrial, commercial and transport units} & \text{Airports} \\
\hline
7 & \text{Artificial surfaces} & \text{Mine, dump and construction sites} & \text{Mineral extraction sites} \\
\hline
8 & \text{Artificial surfaces} & \text{Mine, dump and construction sites} & \text{Dump sites} \\
\hline
9 & \text{Artificial surfaces} & \text{Mine, dump and construction sites} & \text{Construction sites} \\
\hline
10 & \text{Artificial surfaces} & \text{Artificial, non-agricultural vegetated areas} & \text{Green urban areas} \\
\hline
11 & \text{Artificial surfaces} & \text{Artificial, non-agricultural vegetated areas} & \text{Sport and leisure facilities} \\
\hline
12 & \text{Agricultural areas} & \text{Arable land} & \text{Non-irrigated arable land} \\
\hline
13 & \text{Agricultural areas} & \text{Arable land} & \text{Permanently irrigated land} \\
\hline
14 & \text{Agricultural areas} & \text{Arable land} & \text{Rice fields} \\
\hline
15 & \text{Agricultural areas} & \text{Permanent crops} & \text{Vineyards} \\
\hline
16 & \text{Agricultural areas} & \text{Permanent crops} & \text{Fruit trees and berry plantations} \\
\hline
17 & \text{Agricultural areas} & \text{Permanent crops} & \text{Olive groves} \\
\hline
18 & \text{Agricultural areas} & \text{Pastures} & \text{Pastures} \\
\hline
19 & \text{Agricultural areas} & \text{Heterogeneous agricultural areas} & \text{Annual crops associated with permanent crops} \\
\hline
20 & \text{Agricultural areas} & \text{Heterogeneous agricultural areas} & \text{Complex cultivation patterns} \\
\hline
21 & \text{Agricultural areas} & \text{Heterogeneous agricultural areas} & \text{Land principally occupied by agriculture, with significant areas of natural vegetation} \\
\hline
22 & \text{Agricultural areas} & \text{Heterogeneous agricultural areas} & \text{Agro-forestry areas} \\
\hline
23 & \text{Forest and semi natural areas} & \text{Forests} & \text{Broad-leaved forest} \\
\hline
24 & \text{Forest and semi natural areas} & \text{Forests} & \text{Coniferous forest} \\
\hline
25 & \text{Forest and semi natural areas} & \text{Forests} & \text{Mixed forest} \\
\hline
26 & \text{Forest and semi natural areas} & \text{Scrub and/or herbaceous vegetation associations} & \text{Natural grasslands} \\
\hline
27 & \text{Forest and semi natural areas} & \text{Scrub and/or herbaceous vegetation associations} & \text{Moors and heathland} \\
\hline
28 & \text{Forest and semi natural areas} & \text{Scrub and/or herbaceous vegetation associations} & \text{Sclerophyllous vegetation} \\
\hline
29 & \text{Forest and semi natural areas} & \text{Scrub and/or herbaceous vegetation associations} & \text{Transitional woodland-shrub} \\
\hline
30 & \text{Forest and semi natural areas} & \text{Open spaces with little or no vegetation} & \text{Beaches, dunes, sands} \\
\hline
31 & \text{Forest and semi natural areas} & \text{Open spaces with little or no vegetation} & \text{Bare rocks} \\
\hline
32 & \text{Forest and semi natural areas} & \text{Open spaces with little or no vegetation} & \text{Sparsely vegetated areas} \\
\hline
33 & \text{Forest and semi natural areas} & \text{Open spaces with little or no vegetation} & \text{Burnt areas} \\
\hline
34 & \text{Forest and semi natural areas} & \text{Open spaces with little or no vegetation} & \text{Glaciers and perpetual snow} \\
\hline
35 & \text{Wetlands} & \text{Inland wetlands} & \text{Inland marshes} \\
\hline
36 & \text{Wetlands} & \text{Inland wetlands} & \text{Peat bogs} \\
\hline
37 & \text{Wetlands} & \text{Maritime wetlands} & \text{Salt marshes} \\
\hline
38 & \text{Wetlands} & \text{Maritime wetlands} & \text{Salines} \\
\hline
39 & \text{Wetlands} & \text{Maritime wetlands} & \text{Intertidal flats} \\
\hline
40 & \text{Water bodies} & \text{Inland waters} & \text{Water courses} \\
\hline
41 & \text{Water bodies} & \text{Inland waters} & \text{Water bodies} \\
\hline
42 & \text{Water bodies} & \text{Marine waters} & \text{Coastal lagoons} \\
\hline
43 & \text{Water bodies} & \text{Marine waters} & \text{Estuaries} \\
\hline
44 & \text{Water bodies} & \text{Marine waters} & \text{Sea and ocean} \\
\hline
\end{array}



El conjunto de datos se actualiza cada 6 años, por lo que en nuestro caso utilizaremos los datos de 2006, 2012 y 2018. El siguiente gráfico es una visualización de los datos de 2018.

In [30]:
import rasterio
import numpy as np
import pickle
import matplotlib.patches as mpatches

ruta_tiff = "../data/Variables/U2018_CLC2018_V2020_20u1.tif"
custom_colors = pickle.load(open("../data/figures/custom_colors.pkl", "rb"))

with rasterio.open(ruta_tiff) as dataset:
    datos = dataset.read(1)  
datos = datos[:10000, 10000:]
rgb_image = np.zeros((datos.shape[0], datos.shape[1], 3), dtype=np.uint8)
for key, color in custom_colors.items():
    rgb_image[datos == key] = color
plt.figure(figsize=(12, 12))
plt.imshow(rgb_image)
plt.axis('off') 
plt.title("Corine Land Cover año 2018", fontsize=18, pad = 20)
legend_patches = [mpatches.Patch(color=np.array(color)/255, label=str(key)) for key, color in custom_colors.items()]
legend_patches.pop(-1)
plt.legend(handles=legend_patches,
            loc='upper left',
            title="Clases", 
            fontsize=7)
plt.savefig("../data/figures/CLC_plot.png", bbox_inches="tight", dpi=300)
plt.clf()

<Figure size 1200x1200 with 0 Axes>

<div style="text-align: center;">
    <img src="../data/figures/CLC_plot.png" alt="Área quemada en España (2008-2024)" width="800">
</div>

### 2.3 Datos DEM

En cuanto a los datos DEM (Digital Elevation Model), usaremos tres fuentes de datos: la elevación del terreno (metros), la orientación del terreno (grados) y la pendiente (grados). Los datos los obtuvieron la "European Space Agency" y Copernicus, a una resolución de 30m $\times$ 30m. En la siguiente imagen se pueden observar estas tres fuentes de datos graficados en Donostia.

In [31]:
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
import matplotlib.colors as mcolors

dem_file = "../data/Variables/DEM_donostia.tif"
aspect_file = "../data/Variables/Aspect_donostia.tif"
slope_file = "../data/Variables/Slope_donostia.tif"

fig, ax = plt.subplots(3, 1, figsize=(10, 15))

# Open the DEM file using rasterio
with rasterio.open(dem_file) as src:
    dem_data = src.read(1)  # Read the first band
    dem_extent = (src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top)
    im0 = ax[0].imshow(dem_data, cmap='terrain', extent=dem_extent)
    ax[0].set_title("Modelo Digital de Elevación (DEM)")
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].set_ylim(src.bounds.bottom + 0.003, src.bounds.top)
    # Add colorbar for the DEM
    cbar0 = fig.colorbar(im0, ax=ax[0], orientation='vertical', fraction=0.03, pad=0.04)
    cbar0.set_label("Elevation (meters)")

with rasterio.open(aspect_file) as src:
    im1 = show(src, ax=ax[1], title="Orientación")
    ax[1].set_ylim(src.bounds.bottom + 0.003, src.bounds.top)
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    # Add colorbar for the Aspect
    norm = mcolors.Normalize(vmin=src.read(1).min(), vmax=src.read(1).max())
    cbar1 = fig.colorbar(im1.get_images()[0], ax=ax[1], orientation='vertical', fraction=0.03, pad=0.04)
    cbar1.set_label("Orientación (grados)")

with rasterio.open(slope_file) as src:
    im2 = show(src, ax=ax[2], title="Pendiente")
    ax[2].set_ylim(src.bounds.bottom + 0.003, src.bounds.top)
    ax[2].set_xticks([])
    ax[2].set_yticks([])
    # Add colorbar for the Slope
    norm = mcolors.Normalize(vmin=src.read(1).min(), vmax=src.read(1).max())
    cbar2 = fig.colorbar(im2.get_images()[0], ax=ax[2], orientation='vertical', fraction=0.03, pad=0.04)
    cbar2.set_label("Pendiente (grados)")

plt.savefig("../data/figures/DEM_plots.png", bbox_inches="tight", dpi=300)
plt.clf()


<Figure size 1000x1500 with 0 Axes>

<div style="text-align: center;">
    <img src="../data/figures/DEM_plots.png" alt="Área quemada en España (2008-2024)" width="800">
</div>

### 2.4 Distancia a carreteras, distancia a rios, distancia a vías de tren y densidad poblacional

Para introducir el factor humano usaremos estas cuatro variables, algunas creadas mediante el software QGIS:
- Distancia a carreteras
- Distancia a rios
- Distancia a vías de tren
- Densidad poblacional

Esta última cambia con el tiempo y la base de datos de [WorldPop](https://www.worldpop.org/) ofrece datos anuales.

In [32]:
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
import matplotlib.colors as mcolors

railways_file = "../data/Variables/Dist_to_railways_SPAIN.tif"
roads_file = "../data/Variables/Dist_to_roads_SPAIN.tif"
waterways_file = "../data/Variables/Dist_to_waterways_SPAIN.tif"
popdens_file = "../data/Variables/Pop_dens_SPAIN.tif"

fig, ax = plt.subplots(2, 2, figsize=(25, 20))

# Open the DEM file using rasterio
with rasterio.open(popdens_file) as src:
    im1 = show(src, ax=ax[0,0], title="Densidad poblacional (2020)")
    ax[0,0].set_ylim(src.bounds.bottom + 0.05, src.bounds.top-0.05)
    ax[0,0].set_xlim(src.bounds.left + 0.05, src.bounds.right-0.05)
    ax[0,0].set_xticks([])
    ax[0,0].set_yticks([])
    ax[0,0].set_title("Densidad poblacional (2020)", fontsize=25)
    norm = mcolors.Normalize(vmin=src.read(1).min(), vmax=src.read(1).max())
    cbar1 = fig.colorbar(im1.get_images()[0], ax=ax[0,0], orientation='vertical', fraction=0.03, pad=0.04)

with rasterio.open(roads_file) as src:
    im1 = show(src, ax=ax[1,0], title="Distancia a carreteras")
    ax[1,0].set_ylim(src.bounds.bottom + 0.05, src.bounds.top-0.05)
    ax[1,0].set_xlim(src.bounds.left + 0.05, src.bounds.right-0.05)
    ax[1,0].set_xticks([])
    ax[1,0].set_yticks([])
    ax[1,0].set_title("Distancia a carreteras", fontsize=25)
    norm = mcolors.Normalize(vmin=src.read(1).min(), vmax=src.read(1).max())
    cbar1 = fig.colorbar(im1.get_images()[0], ax=ax[1,0], orientation='vertical', fraction=0.03, pad=0.04)

with rasterio.open(waterways_file) as src:
    im2 = show(src, ax=ax[0,1], title="Distancia a rios")
    ax[0,1].set_ylim(src.bounds.bottom + 0.05, src.bounds.top-0.05)
    ax[0,1].set_xlim(src.bounds.left + 0.05, src.bounds.right-0.05)
    ax[0,1].set_xticks([])
    ax[0,1].set_yticks([])
    ax[0,1].set_title("Distancia a rios", fontsize=25)
    norm = mcolors.Normalize(vmin=src.read(1).min(), vmax=src.read(1).max())
    cbar2 = fig.colorbar(im2.get_images()[0], ax=ax[0,1], orientation='vertical', fraction=0.03, pad=0.04)

with rasterio.open(railways_file) as src:
    im2 = show(src, ax=ax[1,1], title="Distancia a vias de tren")
    ax[1,1].set_ylim(src.bounds.bottom + 0.05, src.bounds.top-0.05)
    ax[1,1].set_xlim(src.bounds.left + 0.05, src.bounds.right-0.05)
    ax[1,1].set_xticks([])
    ax[1,1].set_yticks([])
    norm = mcolors.Normalize(vmin=src.read(1).min(), vmax=src.read(1).max())
    cbar2 = fig.colorbar(im2.get_images()[0], ax=ax[1,1], orientation='vertical', fraction=0.03, pad=0.04)
    ax[1,1].set_title("Distancia a vias de tren", fontsize=25)

plt.savefig("../data/figures/Humans_plot.png", bbox_inches="tight", dpi=300)
plt.clf()


<Figure size 2500x2000 with 0 Axes>

<div style="text-align: center;">
    <img src="../data/figures/Humans_plot.png" alt="Área quemada en España (2008-2024)" width="1300">
</div>

### 2.5 Natura2000

La red Natura 2000 es una red ecológica europea de áreas de conservación de la biodiversidad. Introduciremos esta variable porque el conjunto de datos de EFFIS original también la utiliza.

In [33]:
import geopandas as gpd
import contextily as ctx

Natura2000 = gpd.read_file("../data/Variables/N2000_2.geojson")
ax = Natura2000.plot(figsize=(10, 10), alpha=0.5, edgecolor='k', color='lightgreen')
ax.set_title("Red Natura 2000 en España", fontsize=20)
ax.set_xticks([])
ax.set_yticks([])
ctx.add_basemap(ax, crs=Natura2000.crs.to_string())

plt.savefig("../data/figures/Natura2000.png", bbox_inches="tight", dpi=300)
plt.clf()

<Figure size 1000x1000 with 0 Axes>

<div style="text-align: center;">
    <img src="../data/figures/Natura2000.png" alt="Área quemada en España (2008-2024)" width="800">
</div>

## 3. Homogeneización de los datos
En esta sección vamos a explicar brevemente cómo se han homogeneizado los datos. 

### 3.1 Instancias del dataset

Para transformar las bases de datos que hemos presentado en un dataset con datos tabulares, vamos a utilizar celdas espacio-temporales, las cuales serán nuestras instancias que utilizaremos para entrenar los algoritmos. Para ello, hemos creado celdas espaciales que cubren toda España con una resolución espacial de 1km $\times$ 1km. También hemos creado celdas con resolución espacial de 20km $\times$ 20km para visualizarlas.

In [34]:
boxes_visual_20 = gpd.read_file("../data/Variables/Spain_boxes_20km.json")
boxes_visual_1 = gpd.read_file("../data/Variables/Spain_boxes_1km.json")


In [35]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
boxes_visual_20.plot(ax=ax[0], edgecolor='k', color='lightblue')
ax[0].set_title("Celdas de 20km x 20km (para visualizar)", fontsize=20)
ax[0].set_xlim([2.5*10**6, 3.9*10**6])
ax[0].set_ylim([1.5*10**6, 2.5*10**6])
ax[0].set_xticks([])
ax[0].set_yticks([])

boxes_visual_1.plot(ax= ax[1], edgecolor='k', color='red', figsize=(10, 10), alpha=0.1)
ax[1].set_title("Celdas de 1km x 1km", fontsize=20)
ax[1].set_xlim([3.345*10**6, 3.37*10**6])
ax[1].set_ylim([2.31*10**6, 2.33*10**6])
ax[1].set_xticks([])
ax[1].set_yticks([])
ctx.add_basemap(ax[1], crs=boxes_visual_1.crs.to_string())

plt.savefig("../data/figures/Boxes.png", bbox_inches="tight", dpi=300)
plt.clf()

<Figure size 2000x1000 with 0 Axes>

<div style="text-align: center;">
    <img src="../data/figures/Boxes.png" alt="Área quemada en España (2008-2024)" width="1500">
</div>

##### Variables tabulares:

Cada celda de 1km $\times$ 1km tendrá asignadas varias variables tabulares:
- <u>Corine Land Cover</u>: para cada clase, generaremos una variable que será la proporción de casillas de 100m $\times$ 100m de esa clase que hay en la celda. Por lo que el dataset CLC nos dará 44 variables.
- <u>DEM</u>: 
    - <u>Elevación</u>: Para cada celda espacial, calcularemos las siguientes variables de elevación: suma, media , mediana, desviación estandar, mínimo, máximo y rango.
    - <u>Orientación</u>: Dividiremos 360º en 8 clases: (0-45), (45-90), (90-135), (135-180), (180-225), (225-270), (270-315), y (315-360). Para cada clase de orientación, se calculará la proporción de celdas de 30m × 30m que pertenecen a esa clase dentro de la celda de 1km × 1km. Esto nos dará 8 variables adicionales asociadas a la orientación.
    - <u>Pendiente</u>: Para cada celda espacial, calcularemos las siguientes variables de pendiente: suma, media , mediana, desviación estandar, mínimo, máximo y rango.
- <u>Densidad poblacional</u>: Calcularemos el mínimo, el máximo y la media de cada celda espacio-temporal.
- <u>Distancia a carreteras</u>: Calcularemos la media, la mediana, el mínimo y el máximo.
- <u>Distancia a rios</u>: Calcularemos la media, la mediana, el mínimo y el máximo.
- <u>Distancia a vias de tren</u>: Calcularemos la media, la mediana, el mínimo y el máximo.
- <u>Natura 2000</u>: Calcularemos la proporción de terreno de Natura 2000 que hay en cada celda.

Es decir, de estos conjuntos de datos obtenemos 82 variables tabulares. Además, vamos a añadir las siguientes variables:
- <u>Coordenada x</u> (número entero)
- <u>Coordenada y</u> (número entero)
- <u>Mes</u>: Mes en el que pasó el incendio

Estas coordenadas sirven para identificar y ennumerar las celdas (son índices), es decir, la tupla (x,y) identifica a una única celda.

<u>Nota</u>: La homogeneización de las variables y generación del "datacube" se ha hecho mediante el software QGIS: una herramienta que sirve para manejar datos espaciales.

### 3.2 Clase a predecir

#### 3.2.1 Instancias positivas
Una vez que tenemos bien definidas las instancias, la clase a predecir será si hay fuego en una celda un día concreto o no. Utilizando la última base de datos que no hemos usado, EFFIS, generamos las instancias positivas (en las que ha habido un incendio). Esto lo hemos hecho nuevamente utilizando el software QGIS.

In [49]:
import pandas as pd
fire_instances = pd.read_csv("../data/FinalDataset/Fire_instances_expandedbydate.csv")
# fire_instances.drop(columns=["initialdate","finaldate"], inplace=True)
len(fire_instances)

Index(['initialdate', 'finaldate', 'x_index', 'y_index', 'elevation_sum',
       'elevation_mean', 'elevation_median', 'elevation_stdev',
       'elevation_min', 'elevation_max', 'elevation_range', 'slope_sum',
       'slope_mean', 'slope_median', 'slope_stdev', 'slope_min', 'slope_max',
       'slope_range', 'aspect_NODATA', 'aspect_0_45', 'aspect_45_90',
       'aspect_90_135', 'aspect_135_180', 'aspect_180_225', 'aspect_225_270',
       'aspect_270_315', 'aspect_315_360', 'dist_to_road_mean',
       'dist_to_road_median', 'dist_to_road_min', 'dist_to_road_max',
       'dist_to_waterway_mean', 'dist_to_waterway_median',
       'dist_to_waterway_min', 'dist_to_waterway_max', 'dist_to_railway_mean',
       'dist_to_railway_median', 'dist_to_railway_min', 'dist_to_railway_max',
       'natura2000_NODATA', 'natura2000_1', 'CLC_1', 'CLC_2', 'CLC_3', 'CLC_4',
       'CLC_5', 'CLC_6', 'CLC_7', 'CLC_8', 'CLC_9', 'CLC_10', 'CLC_11',
       'CLC_12', 'CLC_13', 'CLC_14', 'CLC_15', 'CLC_16', 'CL

#### 3.2.2 Instancias negativas

El conjunto de datos real debería de tener todas las instancias negativas que ha habido, es decir, todas las celdas durante todos los días que no ha habido fuegos. Sin embargo, esto supondría tener alrededor de 100.000 instancias negativas por cada instancia positiva, por lo que para este trabajo tomaremos instancias negativas aleatoriamente hasta tener una cierta cantidad deseada.

Entonces, para generar las instancias negativas tomaremos días aleatorios que no interfieran con las instancias positivas ya creadas. 

In [ ]:
datacube = gpd.read_file("../data/FinalDataset/Datacube_1.gpkg")

Index(['x_index', 'y_index', 'elevation_sum', 'elevation_mean',
       'elevation_median', 'elevation_stdev', 'elevation_min', 'elevation_max',
       'elevation_range', 'slope_sum',
       ...
       'population_density_2020_mean', 'population_density_2020_min',
       'population_density_2020_max', 'dist_to_railway_mean',
       'dist_to_railway_median', 'dist_to_railway_min', 'dist_to_railway_max',
       'natura2000_NODATA', 'natura2000_1', 'geometry'],
      dtype='object', length=217)

In [ ]:
import random
from datetime import datetime, timedelta
import numpy as np
import warnings

def GenerateBox(date, row):
    """
    Receives as inputs a date and a row from the datacube, 
    
    and returns the values of the row filtered by the selected date.
    """

    year = date.year
    month = date.month

    selected_columns = ["initialdate", "finaldate", "x_index", "y_index",
                        "elevation_sum", "elevation_mean", "elevation_median","elevation_stdev", "elevation_min", "elevation_max", "elevation_range",
                        "slope_sum", "slope_mean", "slope_median", "slope_stdev", "slope_min", "slope_max", "slope_range",
                        "aspect_NODATA", "aspect_0_45", "aspect_45_90", "aspect_90_135", "aspect_135_180", "aspect_180_225", "aspect_225_270", "aspect_270_315", "aspect_315_360",
                        "dist_to_road_mean", "dist_to_road_median", "dist_to_road_min", "dist_to_road_max",
                        "dist_to_waterway_mean", "dist_to_waterway_median", "dist_to_waterway_min", "dist_to_waterway_max",
                        "dist_to_railway_mean", "dist_to_railway_median", "dist_to_railway_min", "dist_to_railway_max",
                        "natura2000_NODATA", "natura2000_1"
                        ]  
    
    # Corine Land Cover
    if year < 2012:
        year_clc = 2006
    elif year < 2018:
        year_clc = 2012
    else:
        year_clc = 2018
    clc_columns = [f"CLC_{year_clc}_{i}" for i in range(1,45)]
    selected_columns = selected_columns + clc_columns + [f"CLC_{year_clc}_NODATA"]

    # Population Density
    if year > 2020:
        year_popdens = 2020
    else:
        year_popdens = year
    popdens_columns = [f"population_density_{year_popdens}_{case}" for case in ["mean", "min", "max"]]
    selected_columns += popdens_columns
    
    filtered_row_values = row[selected_columns].values
    filtered_row_values = np.append(filtered_row_values, [int(month)])

    return filtered_row_values


def generate_random_date(start_date, end_date):
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    return start_date + timedelta(days=random_days)

def check_box(box, fire_instances):
    """
    Receives as inputs a box and returns whether the box has collisions with fire instances.
    """
    pass


n_nofire_instances = 2 * len(fire_instances)
nofire_instances = np.zeros((n_nofire_instances, len(fire_instances.columns)), dtype=object)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

i = 0
while i < len(nofire_instances):
    i_random = random.randint(0,len(datacube)-1) # Select random box
    random_box = datacube.iloc[i_random]
    random_date = generate_random_date(start_date=datetime(2008,1,1),
                                       end_date=datetime(2024,12,31))
    random_box["initialdate"] = random_date.strftime('%Y-%m-%d')
    random_box["finaldate"] = random_date.strftime('%Y-%m-%d')

    random_box_values = GenerateBox(random_date, random_box)
    if check_box(random_box_values, fire_instances):
        nofire_instances[i] = random_box_values
        i += 1
    else:
        continue
    if i % 1000 == 0:
        print(f"Length = {i} / {n_nofire_instances}")

warnings.simplefilter(action='default', category=pd.errors.SettingWithCopyWarning)

Length = 0 / 157084
Length = 1000 / 157084
Length = 2000 / 157084
Length = 3000 / 157084
Length = 4000 / 157084
Length = 5000 / 157084
Length = 6000 / 157084
Length = 7000 / 157084
Length = 8000 / 157084
Length = 9000 / 157084
Length = 10000 / 157084
Length = 11000 / 157084
Length = 12000 / 157084
Length = 13000 / 157084
Length = 14000 / 157084
Length = 15000 / 157084
Length = 16000 / 157084
Length = 17000 / 157084
Length = 18000 / 157084
Length = 19000 / 157084
Length = 20000 / 157084
Length = 21000 / 157084
Length = 22000 / 157084
Length = 23000 / 157084
Length = 24000 / 157084
Length = 25000 / 157084
Length = 26000 / 157084
Length = 27000 / 157084
Length = 28000 / 157084
Length = 29000 / 157084
Length = 30000 / 157084
Length = 31000 / 157084
Length = 32000 / 157084
Length = 33000 / 157084
Length = 34000 / 157084


KeyboardInterrupt: 